In [1]:
import numpy as np
import os
import itertools
import collections
import re
import copy
import math
from abc import ABC, abstractmethod
import inspect

import logging 

# IPython notebooks
import import_ipynb
import dataset_duplicits_toolkit

In [8]:
class RunIteratorSettings:
    def __init__(self
        , zbynek_mode = False # I'm interested in how much I'm loosing
        , remove_duplicits_from_population = True
        , remove_duplicits_from_archive = True
        , remove_already_known_points_from_tss_selection_process = False # aka mistake...
        , remove_already_known_points_from_evaluation_process = True 
        , tss = 2 # avaiable = {0,2}
        , use_distance_weight = True
        ):
        a = locals()
        del a['self']
        self.__dict__.update(a)
        
    def save(self, dest):
        raise NotImplementedError('someone is lazy (－‸ლ)')
        

In [3]:
class TSSbase(ABC):
    def __init__(self
            , population
            , inv_covariance_sqrt):
        self.population = population
        self.inv_covariance_sqrt = inv_covariance_sqrt
        
        assert isinstance(self.inv_covariance_sqrt, np.ndarray)
        assert len(self.inv_covariance_sqrt.shape) == 2
        assert self.inv_covariance_sqrt.shape[0] == self.inv_covariance_sqrt.shape[1]
        assert self.inv_covariance_sqrt.shape[0] > 0
        assert len(self.population.shape) == 2
        assert self.population.shape[1] == self.inv_covariance_sqrt.shape[0]
        
    @abstractmethod
    def __call__(self, archive_points, archive_evaluation, compute_distances=True, add_minimal_distances=False):
        assert len(archive_points.shape) == 2
        assert archive_points.shape[1] == self.population.shape[1]
        assert len(archive_evaluation.shape) == 1
        assert archive_evaluation.shape[0] == archive_points.shape[0]
        
        if compute_distances or add_minimal_distances:
            # (O, G, D)
            differences = self.population[np.newaxis, :, :] -  archive_points[:, np.newaxis, :]
            # (O, G)
            distances = self.mahalanobis_distance(differences, self.inv_covariance_sqrt, do_not_square=True)
        else:
            distances = None
        
        if add_minimal_distances:
            return distances, {'minimal_distances': np.sqrt(np.amin(distances, axis=1))}
        else:
            return distances, {}
        
    @staticmethod
    def apply_mask_to_dictionary(mask, dictionary):
        for key in dictionary.keys():
            data = dictionary[key]
            if isinstance(data, np.ndarray) and len(data.shape) == 1 and len(data) == len(mask):
                dictionary[key] = data[mask]
            else:
                raise NotImplementedError(f'How to convert {key} ?')
        return dictionary
    
    @staticmethod
    def remove_duplicits_basic(points, *others):
        oepm = dataset_duplicits_toolkit.duplicit_mask(points)
        points = points[oepm, ...]
        return (points,) + tuple(x[oepm, ...] for x in others)
    
    @staticmethod
    def mahalanobis_distance(differences, inverse_of_square_of_cov_mat, do_not_square=False):
        N = differences.shape[:-1]
        D = differences.shape[-1]
        
        assert inverse_of_square_of_cov_mat.shape == (D,D)
        
        centered_points = np.matmul(differences, inverse_of_square_of_cov_mat)
        
        if do_not_square:
            return np.sum(np.square(centered_points), axis=-1)
        else:
            return np.sqrt(np.sum(np.square(centered_points), axis=-1))

In [4]:
class TSS0(TSSbase):
    # The hungry one...
    
    def __call__(self, archive_points, archive_evaluation, **kwargs):
        distances, other_info = super().__call__(archive_points, archive_evaluation, \
                                            compute_distances=False, **kwargs)
        mask = np.ones(shape=(len(archive_points),), dtype=np.bool)
        return mask, other_info

In [5]:
class TSS2(TSSbase):
    def __init__(self
            , population
            , inv_covariance_sqrt
            , maximum_distance
            , maximum_number):
        super().__init__(population, inv_covariance_sqrt)
        
        self.maximum_distance = maximum_distance
        self.maximum_number = maximum_number
        assert isinstance(self.maximum_number, (int,float))
        assert isinstance(self.maximum_distance, (int,float))
        
    def __call__(self, archive_points, archive_evaluation, **kwargs):
        distances, other_info = super().__call__(archive_points, archive_evaluation, **kwargs)
        
        D = archive_points.shape[1]
        N = archive_points.shape[0]
        P = self.population.shape[0]
        
        # Firstly, do not limit number of neighbours
        mask = np.any(distances <= self.maximum_distance**2, axis=1)
        if np.sum(mask) <= self.maximum_number:
            return mask, self.apply_mask_to_dictionary(mask, other_info)
        
        # It's too much
        distances_indices = np.argsort(distances, axis=0)
        distances_sorted = np.stack(
            [distances[distances_indices[:,i],i] for i in range(P)], axis=1)
        distances_enable  = distances_sorted <= self.maximum_distance**2
        
        # compute minimal size
        min_perin_size = math.floor(self.maximum_number / P)
        
        mask.fill(False) # reuse mask array
        new_elems_ind = distances_indices[:min_perin_size,:][distances_enable[:min_perin_size,:]]
        mask[new_elems_ind] = True
        enabled_n = np.sum(mask)
        
        if enabled_n == self.maximum_number:
            return mask, self.apply_mask_to_dictionary(mask, other_info)
        else:
            # create another array
            new_mask = mask.copy()
        
        for perinp in itertools.count(start=min_perin_size):
            new_elems_ind = np.unique(distances_indices[perinp, :][distances_enable[perinp, :]])
            new_elems_ind = new_elems_ind[new_mask[new_elems_ind] == False]
            
            new_mask[new_elems_ind] = True
            
            enabled_o = enabled_n
            enabled_n += len(new_elems_ind) #np.sum(new_mask)
            
            if enabled_n == self.maximum_number:
                return new_mask, self.apply_mask_to_dictionary(new_mask, other_info)
            elif enabled_n < self.maximum_number:
                mask, new_mask = new_mask, mask
                np.copyto(new_mask, mask)
            else:
                new_elems_eval = archive_evaluation[new_elems_ind]
                new_elems_ind_selection = np.argsort(new_elems_eval)[:self.maximum_number - enabled_o]
                selection = new_elems_ind[new_elems_ind_selection]
                
                mask[selection] = True
                return mask, self.apply_mask_to_dictionary(mask, other_info)

In [6]:
class State:
    def __init__(self, run, gen):
        self._run = run
        self._settings = run.settings
        self._gen = gen
        
        if gen+2 == len(self._run.gen_split):
            pos, next_pos = self._run.gen_split[gen], None
        else:
            pos, next_pos = self._run.gen_split[gen:gen+2]
        
        orig_evaled_mask = self._run.orig_evaled[:pos]
        self.x_fit = self._run.points[:pos, ...][orig_evaled_mask, ...]
        self.y_fit = self._run.fvalues_orig[:pos][orig_evaled_mask]
        
        population = self._run.points[pos:next_pos]
        
        # Duplicit filtering
        if self._settings.remove_duplicits_from_archive:
            self.x_fit, self.y_fit = self.remove_duplicits_basic(self.x_fit, self.y_fit)
            
        self.archive_points = self.x_fit
            
        if self._settings.remove_duplicits_from_population:    
            population, = self.remove_duplicits_basic(population)
        
        if self._settings.remove_already_known_points_from_tss_selection_process:
            population = self.remove_elements(self.x_fit, population)
            
        # TSS filtering
        tss = self.get_tss(population)
        self.x_fit, self.y_fit = self.select_by_tss(tss, self.x_fit, self.y_fit)
        
        # Evaluation ....
        self.x_eval = self._run.points[pos:next_pos, ...]
        self.y_eval = self._run.fvalues_orig[pos:next_pos]
        
        if self._settings.zbynek_mode: 
            orig = self._run.orig_evaled[pos:next_pos]
            
            self.x_eval = self.x_eval[~orig, ...]
            self.y_eval = self.y_eval[~orig]
            self.y_eval_base = self._run.fvalues[pos:next_pos][~orig]
        
        # Remove evaluation dupl.
        if self._settings.remove_already_known_points_from_evaluation_process:
            if self._settings.zbynek_mode: 
                self.x_eval, self.y_eval, self.y_eval_base = \
                    self.remove_elements(self.archive_points, self.x_eval, self.y_eval, self.y_eval_base)
            else:
                self.x_eval, self.y_eval = self.remove_elements(self.archive_points, self.x_eval, self.y_eval)
            
        
    @staticmethod
    def remove_duplicits_basic(points, *others):
        oepm = dataset_duplicits_toolkit.duplicit_mask(points)
        points = points[oepm, ...]
        return (points,) + tuple(x[oepm, ...] for x in others)
    
    @staticmethod
    def remove_elements(to_remove, points, *others):
        assert to_remove.shape[1] == points.shape[1]
        assert len(to_remove.shape) == len(points.shape) == 2
        
        all_elems = np.concatenate([to_remove, points], axis=0)
        hot_mask = np.concatenate([
            np.ones(shape=(len(to_remove),), dtype=np.bool),
            np.zeros(shape=(len(points),), dtype=np.bool)
            ])
        mask = dataset_duplicits_toolkit.duplicit_mask_with_hot_elements(all_elems, hot_mask)[len(to_remove):]
        return (points[mask, ...], ) + tuple(x[mask, ...] for x in others)
        
    def select_by_tss(self, tss, points, evals, *others):
        kwargs = {}
        if self._settings.use_distance_weight:
            kwargs['add_minimal_distances'] = True
            
        mask, other_info = tss(points, evals, **kwargs)
        
        if self._settings.use_distance_weight:
            self.distances = other_info['minimal_distances']
        
        return (points[mask, ...], evals[mask]) + tuple(x[mask, ...] for x in others)
        
    def get_tss(self, population):
        assert self._run.surrogate_param_type == 'nearest'
        
        inv_cov_sqrt = 1./self._run.surrogate_data_sigmas[self._gen] * \
            np.linalg.inv(self._run.surrogate_data_bds[self._gen, ...]) 
        
        if self._settings.tss == 2:
            maximum_distance = float(self._run.surrogate_param_range)
            
            maximum_number = str(self._run.surrogate_param_set_size_max)
            v = re.match(r'(\d+)\*dim', maximum_number)
            if not v:
                raise NotImplementedError(f"maximum_number channot be interpreted: {maximum_number}")
            maximum_number = int(int(v.group(1))*self._run.dimensions)

            return TSS2(population, inv_cov_sqrt, maximum_distance, maximum_number)
        elif self._settings.tss == 0:
            return TSS0(population, inv_cov_sqrt)
        else:
            raise NotImplementedError("I don't know what to say...")
        

In [7]:
class StateIterator: # / == Run
    '''
        name 
        path
        
        points - points in domain
        fvalues - points in codomain that are assumed to be true
        orig_evaled - what points are exactly evaluated and what are assumed to be true (surrogate)
        gen_split - 
        iruns - 
        evals - the over all number of evaluations of the original fitness function (integer).
        coco - ground truth for codomain
        ...
    '''
    def __init__(self, name=None, path=None, **kwargs):
        self.name = name
        self.path = path
        self.__dict__.update(kwargs)
        self._loaded = False
        
    def __getattr__(self, name):
        # lazy evaluation :)
        if name == "__setstate__":
            raise AttributeError(name)
        if not self._loaded:
            #print('Loading...')
            npob = np.load(self.path)
            self.__dict__.update(npob.items())
            self._loaded = True
            
            self.evals = int(self.evals)
        return super().__getattribute__(name)
    
    def __iter__(self):
        return iter(map(lambda g: State(self, g), range(1,len(self.gen_split)-1)))
        
    def __repr__(self):
        return self.name
    

In [9]:
class RunIterator: # == File
    def __init__(self
        , settings
        , data_folder='../../npz-data'
        , filters = None 
        ):
        self.data_folder = data_folder
        self.settings = settings
        
        if filters is None:
            self.filters = []
        elif isinstance(filters, collections.abc.Iterable):
            self.filters = filters
        else:
            self.filters = [filters]
        
        self._inspect_data_folder()
    
    @staticmethod
    def _order_key(file):
        return [int(num) for num in re.findall("\d+", file)]
                    
    def _inspect_data_folder(self):
        self.data_files = os.listdir(self.data_folder)
        self.data_files = list(self.data_files)
        self.data_files.sort(key=self._order_key)
        
    def __iter__(self):
        instance = copy.deepcopy(self)
        return instance
    
    def __next__(self):
        try:
            while True:
                name = self.data_files.pop(0)
                path = self.data_folder + '/' + name
                obj = StateIterator(name=name, path=path, settings=self.settings)
                
                ok = True
                for fce in self.filters:
                    if not fce(obj):
                        ok = False
                        break
                if not ok:
                    continue
                return obj
        except IndexError:
            raise StopIteration()
        
_ = '''
USAGE: 
iterator = RunIterator(filters=
                        [lambda x: '2D' in x.name,
                         lambda x: x.evals != np.sum(x.orig_evaled)])
for a in itertools.islice(iterator, 10):
    print(f'{a.name}: {a.evals} {np.sum(a.orig_evaled)}')
    u, i, c = np.unique(a.points, axis=0, return_counts=True, return_index=True)
'''

In [10]:
if True:
    settings = RunIteratorSettings(tss=0)
    iterator = RunIterator(settings)
    
    for run in itertools.islice(iterator, 10):
        for sid, state in enumerate(run):
            
            print(f"------------ state {state._gen} {run.points.shape} ---------------")
            for attr in dir(state):
                if isinstance(getattr(state, attr), np.ndarray):
                    print(f"state.{attr} - {getattr(state, attr).shape}")

------------ state 1 (8737, 2) ---------------
state.archive_points - (6, 2)
state.distances - (6,)
state.x_eval - (12, 2)
state.x_fit - (6, 2)
state.y_eval - (12,)
state.y_fit - (6,)
------------ state 2 (8737, 2) ---------------
state.archive_points - (8, 2)
state.distances - (8,)
state.x_eval - (12, 2)
state.x_fit - (8, 2)
state.y_eval - (12,)
state.y_fit - (8,)
------------ state 3 (8737, 2) ---------------
state.archive_points - (8, 2)
state.distances - (8,)
state.x_eval - (12, 2)
state.x_fit - (8, 2)
state.y_eval - (12,)
state.y_fit - (8,)
------------ state 4 (8737, 2) ---------------
state.archive_points - (9, 2)
state.distances - (9,)
state.x_eval - (12, 2)
state.x_fit - (9, 2)
state.y_eval - (12,)
state.y_fit - (9,)
------------ state 5 (8737, 2) ---------------
state.archive_points - (10, 2)
state.distances - (10,)
state.x_eval - (12, 2)
state.x_fit - (10, 2)
state.y_eval - (12,)
state.y_fit - (10,)
------------ state 6 (8737, 2) ---------------
state.archive_points - (12, 2

------------ state 139 (8737, 2) ---------------
state.archive_points - (130, 2)
state.distances - (130,)
state.x_eval - (12, 2)
state.x_fit - (130, 2)
state.y_eval - (12,)
state.y_fit - (130,)
------------ state 140 (8737, 2) ---------------
state.archive_points - (132, 2)
state.distances - (132,)
state.x_eval - (12, 2)
state.x_fit - (132, 2)
state.y_eval - (12,)
state.y_fit - (132,)
------------ state 141 (8737, 2) ---------------
state.archive_points - (132, 2)
state.distances - (132,)
state.x_eval - (12, 2)
state.x_fit - (132, 2)
state.y_eval - (12,)
state.y_fit - (132,)
------------ state 142 (8737, 2) ---------------
state.archive_points - (133, 2)
state.distances - (133,)
state.x_eval - (12, 2)
state.x_fit - (133, 2)
state.y_eval - (12,)
state.y_fit - (133,)
------------ state 143 (8737, 2) ---------------
state.archive_points - (134, 2)
state.distances - (134,)
state.x_eval - (12, 2)
state.x_fit - (134, 2)
state.y_eval - (12,)
state.y_fit - (134,)
------------ state 144 (8737, 

------------ state 238 (8737, 2) ---------------
state.archive_points - (230, 2)
state.distances - (230,)
state.x_eval - (15, 2)
state.x_fit - (230, 2)
state.y_eval - (15,)
state.y_fit - (230,)
------------ state 239 (8737, 2) ---------------
state.archive_points - (230, 2)
state.distances - (230,)
state.x_eval - (18, 2)
state.x_fit - (230, 2)
state.y_eval - (18,)
state.y_fit - (230,)
------------ state 240 (8737, 2) ---------------
state.archive_points - (230, 2)
state.distances - (230,)
state.x_eval - (19, 2)
state.x_fit - (230, 2)
state.y_eval - (19,)
state.y_fit - (230,)
------------ state 241 (8737, 2) ---------------
state.archive_points - (230, 2)
state.distances - (230,)
state.x_eval - (20, 2)
state.x_fit - (230, 2)
state.y_eval - (20,)
state.y_fit - (230,)
------------ state 242 (8737, 2) ---------------
state.archive_points - (230, 2)
state.distances - (230,)
state.x_eval - (17, 2)
state.x_fit - (230, 2)
state.y_eval - (17,)
state.y_fit - (230,)
------------ state 243 (8737, 

------------ state 305 (8737, 2) ---------------
state.archive_points - (313, 2)
state.distances - (313,)
state.x_eval - (21, 2)
state.x_fit - (313, 2)
state.y_eval - (21,)
state.y_fit - (313,)
------------ state 306 (8737, 2) ---------------
state.archive_points - (314, 2)
state.distances - (314,)
state.x_eval - (24, 2)
state.x_fit - (314, 2)
state.y_eval - (24,)
state.y_fit - (314,)
------------ state 307 (8737, 2) ---------------
state.archive_points - (316, 2)
state.distances - (316,)
state.x_eval - (24, 2)
state.x_fit - (316, 2)
state.y_eval - (24,)
state.y_fit - (316,)
------------ state 308 (8737, 2) ---------------
state.archive_points - (319, 2)
state.distances - (319,)
state.x_eval - (23, 2)
state.x_fit - (319, 2)
state.y_eval - (23,)
state.y_fit - (319,)
------------ state 309 (8737, 2) ---------------
state.archive_points - (320, 2)
state.distances - (320,)
state.x_eval - (24, 2)
state.x_fit - (320, 2)
state.y_eval - (24,)
state.y_fit - (320,)
------------ state 310 (8737, 

------------ state 374 (8737, 2) ---------------
state.archive_points - (415, 2)
state.distances - (415,)
state.x_eval - (44, 2)
state.x_fit - (415, 2)
state.y_eval - (44,)
state.y_fit - (415,)
------------ state 375 (8737, 2) ---------------
state.archive_points - (418, 2)
state.distances - (418,)
state.x_eval - (36, 2)
state.x_fit - (418, 2)
state.y_eval - (36,)
state.y_fit - (418,)
------------ state 376 (8737, 2) ---------------
state.archive_points - (420, 2)
state.distances - (420,)
state.x_eval - (46, 2)
state.x_fit - (420, 2)
state.y_eval - (46,)
state.y_fit - (420,)
------------ state 377 (8737, 2) ---------------
state.archive_points - (422, 2)
state.distances - (422,)
state.x_eval - (38, 2)
state.x_fit - (422, 2)
state.y_eval - (38,)
state.y_fit - (422,)
------------ state 378 (8737, 2) ---------------
state.archive_points - (422, 2)
state.distances - (422,)
state.x_eval - (42, 2)
state.x_fit - (422, 2)
state.y_eval - (42,)
state.y_fit - (422,)
------------ state 379 (8737, 

...

------------ state 91 (3529, 2) ---------------
state.archive_points - (133, 2)
state.distances - (133,)
state.x_eval - (24, 2)
state.x_fit - (133, 2)
state.y_eval - (24,)
state.y_fit - (133,)
------------ state 92 (3529, 2) ---------------
state.archive_points - (135, 2)
state.distances - (135,)
state.x_eval - (24, 2)
state.x_fit - (135, 2)
state.y_eval - (24,)
state.y_fit - (135,)
------------ state 93 (3529, 2) ---------------
state.archive_points - (137, 2)
state.distances - (137,)
state.x_eval - (24, 2)
state.x_fit - (137, 2)
state.y_eval - (24,)
state.y_fit - (137,)
------------ state 94 (3529, 2) ---------------
state.archive_points - (139, 2)
state.distances - (139,)
state.x_eval - (24, 2)
state.x_fit - (139, 2)
state.y_eval - (24,)
state.y_fit - (139,)
------------ state 95 (3529, 2) ---------------
state.archive_points - (141, 2)
state.distances - (141,)
state.x_eval - (24, 2)
state.x_fit - (141, 2)
state.y_eval - (24,)
state.y_fit - (141,)
------------ state 96 (3529, 2) ---

------------ state 3 (805, 2) ---------------
state.archive_points - (9, 2)
state.distances - (9,)
state.x_eval - (12, 2)
state.x_fit - (9, 2)
state.y_eval - (12,)
state.y_fit - (9,)
------------ state 4 (805, 2) ---------------
state.archive_points - (9, 2)
state.distances - (9,)
state.x_eval - (12, 2)
state.x_fit - (9, 2)
state.y_eval - (12,)
state.y_fit - (9,)
------------ state 5 (805, 2) ---------------
state.archive_points - (11, 2)
state.distances - (11,)
state.x_eval - (12, 2)
state.x_fit - (11, 2)
state.y_eval - (12,)
state.y_fit - (11,)
------------ state 6 (805, 2) ---------------
state.archive_points - (11, 2)
state.distances - (11,)
state.x_eval - (12, 2)
state.x_fit - (11, 2)
state.y_eval - (12,)
state.y_fit - (11,)
------------ state 7 (805, 2) ---------------
state.archive_points - (13, 2)
state.distances - (13,)
state.x_eval - (12, 2)
state.x_fit - (13, 2)
state.y_eval - (12,)
state.y_fit - (13,)
------------ state 8 (805, 2) ---------------
state.archive_points - (14,

------------ state 41 (3949, 2) ---------------
state.archive_points - (80, 2)
state.distances - (80,)
state.x_eval - (12, 2)
state.x_fit - (80, 2)
state.y_eval - (12,)
state.y_fit - (80,)
------------ state 42 (3949, 2) ---------------
state.archive_points - (81, 2)
state.distances - (81,)
state.x_eval - (12, 2)
state.x_fit - (81, 2)
state.y_eval - (12,)
state.y_fit - (81,)
------------ state 43 (3949, 2) ---------------
state.archive_points - (82, 2)
state.distances - (82,)
state.x_eval - (12, 2)
state.x_fit - (82, 2)
state.y_eval - (12,)
state.y_fit - (82,)
------------ state 44 (3949, 2) ---------------
state.archive_points - (83, 2)
state.distances - (83,)
state.x_eval - (12, 2)
state.x_fit - (83, 2)
state.y_eval - (12,)
state.y_fit - (83,)
------------ state 45 (3949, 2) ---------------
state.archive_points - (84, 2)
state.distances - (84,)
state.x_eval - (12, 2)
state.x_fit - (84, 2)
state.y_eval - (12,)
state.y_fit - (84,)
------------ state 46 (3949, 2) ---------------
state.a

------------ state 126 (3949, 2) ---------------
state.archive_points - (187, 2)
state.distances - (187,)
state.x_eval - (12, 2)
state.x_fit - (187, 2)
state.y_eval - (12,)
state.y_fit - (187,)
------------ state 127 (3949, 2) ---------------
state.archive_points - (188, 2)
state.distances - (188,)
state.x_eval - (12, 2)
state.x_fit - (188, 2)
state.y_eval - (12,)
state.y_fit - (188,)
------------ state 128 (3949, 2) ---------------
state.archive_points - (189, 2)
state.distances - (189,)
state.x_eval - (12, 2)
state.x_fit - (189, 2)
state.y_eval - (12,)
state.y_fit - (189,)
------------ state 129 (3949, 2) ---------------
state.archive_points - (189, 2)
state.distances - (189,)
state.x_eval - (12, 2)
state.x_fit - (189, 2)
state.y_eval - (12,)
state.y_fit - (189,)
------------ state 130 (3949, 2) ---------------
state.archive_points - (201, 2)
state.distances - (201,)
state.x_eval - (12, 2)
state.x_fit - (201, 2)
state.y_eval - (12,)
state.y_fit - (201,)
------------ state 131 (3949, 

------------ state 192 (3949, 2) ---------------
state.archive_points - (297, 2)
state.distances - (297,)
state.x_eval - (24, 2)
state.x_fit - (297, 2)
state.y_eval - (24,)
state.y_fit - (297,)
------------ state 193 (3949, 2) ---------------
state.archive_points - (300, 2)
state.distances - (300,)
state.x_eval - (24, 2)
state.x_fit - (300, 2)
state.y_eval - (24,)
state.y_fit - (300,)
------------ state 194 (3949, 2) ---------------
state.archive_points - (302, 2)
state.distances - (302,)
state.x_eval - (24, 2)
state.x_fit - (302, 2)
state.y_eval - (24,)
state.y_fit - (302,)
------------ state 195 (3949, 2) ---------------
state.archive_points - (304, 2)
state.distances - (304,)
state.x_eval - (24, 2)
state.x_fit - (304, 2)
state.y_eval - (24,)
state.y_fit - (304,)
------------ state 196 (3949, 2) ---------------
state.archive_points - (306, 2)
state.distances - (306,)
state.x_eval - (24, 2)
state.x_fit - (306, 2)
state.y_eval - (24,)
state.y_fit - (306,)
------------ state 197 (3949, 

In [11]:
str(run.surrogate_param_set_size_max)
run.dimensions
print(run)
print(state.distances)

exp_doubleEC_28_log_nonadapt_results_1_2D_2_4.npz
[5.24623980e+03 1.14042704e+03 5.82205387e+03 5.01033225e+03
 5.11036491e+03 7.13418604e+03 9.85749544e+03 2.32744977e+03
 2.39753141e+03 4.95929456e+03 8.21725585e+02 1.67592972e+03
 1.59683256e+03 1.85218440e+03 1.69905975e+03 6.34832171e+01
 3.34299153e+02 6.13803695e+02 1.20786136e+02 3.29355069e+02
 3.12502038e+02 4.98963599e+01 2.39155417e+01 1.52161007e+02
 2.62347391e+01 1.52936693e+02 8.05765422e+01 8.05228990e+01
 1.05002012e+02 2.66114590e+01 5.44239781e+01 2.02209489e+02
 1.25677147e+01 4.98010757e+00 9.71624112e+00 1.34883979e+01
 3.12819585e+01 2.30802475e+01 3.72882306e+01 3.44485306e+01
 3.08324729e+01 5.15659144e+01 4.94283447e+01 8.67931588e+01
 1.23222769e+02 7.26581439e+01 1.97173947e+01 1.16365524e+01
 2.84742664e+01 8.87330014e-01 1.49969228e+00 3.02285644e+00
 2.34249637e+00 4.17185472e+00 6.46127163e+00 6.77761187e+00
 7.62329142e+00 7.30017118e+00 1.14047976e+01 1.71017404e+01
 1.29567649e+01 4.04106707e-01 6.87

In [12]:
# TESTS


if __name__ == '__main__':
    from IPython.core.debugger import set_trace
    import unittest
    import scipy
    import scipy.linalg
    import random
    
    def bisect_left(func_less, lo, hi):
        while lo < hi:
            mid = (lo+hi)//2
            # Use __lt__ to match the logic in list.sort() and in heapq
            if func_less(mid): 
                lo = mid+1
            else: 
                hi = mid
        return lo

    
    class TestTSS2(unittest.TestCase):
        def call_tss2(self, generation, points, evaluations, maximum_distance=None, maximum_elements=None, inv_cov_matr_sqrt=None):
            if inv_cov_matr_sqrt is None:
                inv_cov_matr_sqrt = np.eye(generation.shape[1])
            if maximum_elements is None:
                maximum_elements = points.shape[0]
            if maximum_distance is None:
                maximum_distance = np.inf
                
            tss2 = TSS2(generation, inv_cov_matr_sqrt, maximum_distance, maximum_elements)
            mask, other_stuff = tss2(points, evaluations)
            return mask
        
        def test_maximum_elements(self):
            maximum_elements = 30
            
            for elements in range(100):
                generation = np.random.rand(10,2)
                points = np.random.rand(elements, 2)
                evals= np.random.rand(elements,)
                
                mask = self.call_tss2(generation, points, evals, maximum_elements=maximum_elements)
                
                self.assertEqual(np.sum(mask), min(elements, maximum_elements))
                self.assertEqual(len(mask.shape), 1)
                self.assertEqual(mask.shape[0], elements)
                
        def test_maximum_distance_without_limit(self):
            maximum_distance = 0.2
            generation = np.array([
                [0.2, 0.5],
                [0.8, 0.5]
            ])
            points = np.random.rand(100000, 2)
            evals  = np.random.rand(100000)

            mask = self.call_tss2(generation, points, evals, maximum_distance=maximum_distance)
            
            true_mask = np.minimum(
                np.sum(np.square(points - generation[0,:]), axis=1),
                np.sum(np.square(points - generation[1,:]), axis=1)
                ) <= maximum_distance**2
            
            diff = mask ^ true_mask
            self.assertFalse(np.any(diff))
            
        def test_maximum_distance_with_limit_nonoverlaping(self):
            maximum_distance = 0.2
            generation = np.array([
                [0.2, 0.5],
                [0.8, 0.5]
            ])
            
            points = np.random.rand(10000, 2)
            evals  = np.random.rand(10000)
            
            dist_left = np.sqrt(np.sum(np.square(points - generation[0,:]), axis=1))
            dist_right = np.sqrt(np.sum(np.square(points - generation[1,:]), axis=1))
            
            avail = np.sum(np.logical_or(dist_left <= maximum_distance, dist_right <= maximum_distance))
            
            ord_left = np.argsort(dist_left)
            ord_right = np.argsort(dist_right)
            ord_left = ord_left[:np.sum(dist_left <= maximum_distance)]
            ord_right = ord_right[:np.sum(dist_right <= maximum_distance)]
            
            for elems in [random.randint(1,min(ord_left.size, ord_right.size)- 1) for _ in range(10)]:
                mask = self.call_tss2(generation, points, evals, 
                            maximum_elements=elems,
                            maximum_distance=maximum_distance)
                
                cor = np.zeros_like(mask)
                cor[ord_left[:elems//2]] = True
                cor[ord_right[:elems//2]] = True
                    
                if elems % 2 == 1:
                    if evals[ord_left[elems//2]] < evals[ord_right[elems//2]]:
                        cor[ord_left[elems//2]] = True
                    else:
                        cor[ord_right[elems//2]] = True
                        
                diff = mask ^ cor
                self.assertFalse(np.any(diff))
                
        def test_maximum_distance_with_limit_overl(self):
            maximum_distance = 0.8
            generation = np.array([
                [0.2, 0.5],
                [0.8, 0.5]
            ])
            
            points = np.random.rand(10000, 2)
            evals  = np.random.rand(10000)
            
            dist_left = np.sqrt(np.sum(np.square(points - generation[0,:]), axis=1))
            dist_right = np.sqrt(np.sum(np.square(points - generation[1,:]), axis=1))
            
            ord_left = np.argsort(dist_left)
            ord_right = np.argsort(dist_right)
            # outliers - remove
            ord_left = ord_left[:np.sum(dist_left <= maximum_distance)]
            ord_right = ord_right[:np.sum(dist_right <= maximum_distance)]
            
            max_com = min(ord_left.size, ord_right.size)
            # najdi takovy element, ktery je v obou pri dane vzdalenosti
            # ??? TODO -> min vzd.
            
            def func(mid):
                return not bool(len(set(iter(ord_left[:mid])).intersection(set(iter(ord_right[:mid])))))
                
            min_com = bisect_left(func, 0, max_com)
            assert ord_left[min_com-1] in ord_right[:min_com] or \
                   ord_right[min_com-1] in ord_left[:min_com]
            
            
            for elems in [random.randint(min_com,max_com) for _ in range(10)]:
                mask = self.call_tss2(generation, points, evals, 
                                      maximum_distance=maximum_distance, 
                                      maximum_elements=elems)
                n_mask = np.sum(mask)
                mask_arg = set(iter(np.argwhere(mask).flatten()))
                
                # number
                self.assertEqual(n_mask, elems)
                
                # distance
                selected_elements_distances = np.minimum(dist_left[mask], dist_right[mask])
                self.assertLessEqual(np.amax(selected_elements_distances), maximum_distance)
                
                # distance order
                def func(mid):
                    return mask_arg >= set(np.concatenate((ord_left[:mid], ord_right[:mid])))
                
                first_nonres = bisect_left(func, 0, len(mask_arg))
                
                # 1. check if all are ok / fist better (fittness) is ok
                almask = np.zeros_like(mask)
                #almask[
                #    np.unique(np.concatenate((ord_left[:first_nonres-1], ord_right[:first_nonres-1])))
                #] = True
                almask[ np.concatenate((ord_left[:first_nonres-1], ord_right[:first_nonres-1])) ] = True
                
                assert np.sum(almask) <= elems
                
                g = almask ^ mask
                ng = np.sum(g)
                
                self.assertLessEqual(ng,1)

                    
                if ng > 0:
                    if evals[ord_left[first_nonres-1]] < evals[ord_right[first_nonres-1]]:
                        addi = ord_left[first_nonres-1]
                    else:
                        addi = ord_right[first_nonres-1]
                        
                    self.assertTrue(g[addi])
                    self.assertFalse(np.any(g[:addi]))
                    self.assertFalse(np.any(g[addi+1:]))
                    
if __name__ == '__main__':
    class TestState(unittest.TestCase):
        def test_mahalanobis_distance_determ(self):
            C = np.array([[1, 0.5, 0.5], [0.5, 1, 0.5], [0.5, 0.5, 1]])
            C = scipy.linalg.sqrtm(C)
            # C is <cov mat.>^-1/2
            
            mu = np.array([0., 1., 0.])
            points = np.array([[1.,0.,0.]])
            
            # 1. test
            res = TSS2.mahalanobis_distance(points - mu, C)
            self.assertEqual(res.shape, (1,))
            res = res[0]
            self.assertAlmostEqual(res, 1.0)
            
            # 2. test
            points = np.array([[0, 2., 0.]])
            
            res = TSS2.mahalanobis_distance(points - mu, C)
            self.assertEqual(res.shape, (1,))
            res = res[0]
            self.assertAlmostEqual(res, 1.0)
            
            # 3. test
            points = np.array([
                [1., 0., 0.],
                [0., 2., 0.],
                [2., 0., 0.]
            ])
            
            res = TSS2.mahalanobis_distance(points - mu, C)
            self.assertEqual(res.shape, (3,))
            for res_or, res_my in zip([1,1,1.7320508075688772], res):
                self.assertAlmostEqual(res_or, res_my)
                
        def test_mahalanobis_multiple_dimensions(self):
            C = np.array([[1, 0.5, 0.5], [0.5, 1, 0.5], [0.5, 0.5, 1]])
            C = scipy.linalg.sqrtm(C)
            # C is <cov mat.>^-1/2
            
            mu = np.array([0., 1., 0.])
            differences = np.array([
                [
                    [1., 0., 0.],
                    [0., 2., 0.],
                    [2., 0., 0.]
                ],[
                    [1., 0., 0.],
                    [0., 2., 0.],
                    [1., 0., 0.]
                ],[
                    [2., 0., 0.],
                    [0., 2., 0.],
                    [2., 0., 0.]
                ]
            ]) - mu[np.newaxis, np.newaxis, :]
            
            res_ok = np.array([
                [1,1,1.7320508075688772],
                [1,1,1],
                [1.7320508075688772,1,1.7320508075688772]
            ])
            res = TSS2.mahalanobis_distance(differences, C)
            
            self.assertTrue(res.shape == (3,3))
            for a,b in zip(res.flatten(), res_ok.flatten()):
                 self.assertAlmostEqual(a,b)
                
if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False, verbosity=2)   


test_mahalanobis_distance_determ (__main__.TestState) ... ok
test_mahalanobis_multiple_dimensions (__main__.TestState) ... ok
test_maximum_distance_with_limit_nonoverlaping (__main__.TestTSS2) ... ok
test_maximum_distance_with_limit_overl (__main__.TestTSS2) ... ok
test_maximum_distance_without_limit (__main__.TestTSS2) ... ok
test_maximum_elements (__main__.TestTSS2) ... ok

----------------------------------------------------------------------
Ran 6 tests in 0.484s

OK
